In [1]:
import ast
import os
import pandas as pd
import platform
import selenium
import time
import unidecode

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.wait import WebDriverWait
from unidecode import unidecode
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
mdf = pd.read_csv('musicas.csv')
mdf.sample(5)

,indice_musica,titulo_musica,nome_artista,artistas_participacao,letra,album,data_lancamento,tags,identificador
439,439,Olho Por Olho,Beth Carvalho,[],A justiça dos homens\nCondena a bigamia\nNenhu...,Nos Botequins da Vida (1977),1977,"Pop,MPB (Música Popular Brasileira),Samba,Bras...",Olho Por Olho Beth Carvalho
105,105,Ainda Bem,Marisa Monte,[],"[Letra de ""Ainda Bem"" com Marisa Monte]\n\n[Ve...",O Que Você Quer Saber de Verdade (2011),"Oct. 31, 2011","Pop,Brasil,MPB (Música Popular Brasileira),Em ...",Ainda Bem Marisa Monte
66,66,A Palo Seco,Belchior,[],"[Letra de ""A Palo Seco""]\n\n[Verso 1]\nSe você...",Belchior (1974),1974,"Pop,MPB (Música Popular Brasileira),Brasil,Em ...",A Palo Seco Belchior
107,107,Travessia,Milton Nascimento,[],"[Letras de ""Travessia"" com Milton Nascimento]\...",Milton Nascimento / Travessia (1967),1967,"Country,Pop,Folk,Latin Music,MPB (Música Popul...",Travessia Milton Nascimento
363,363,"Estácio, Holly Estácio",Luiz Melodia,[],"[Letra de ""Estácio, Holly Estácio""]\n\n[Verso ...",Pérola Negra (1973),1973,"Pop,Singer-Songwriter,MPB (Música Popular Bras...","Estácio, Holly Estácio Luiz Melodia"


In [3]:
# Quantidade de artistas distintos no CSV

mdf['nome_artista'].nunique()

245

In [4]:
# Função que recebe um nome e retorna o link do perfil do artista correspondente no Genius 

def get_link_musicas_artista(nome_artista):
    nome_normalizado = unidecode(nome_artista).lower().capitalize().replace(' ', '-')
    link_musicas_artista = f'https://genius.com/artists/{nome_normalizado}/songs'
    
    return link_musicas_artista

In [5]:
# Criando um DF apenas com o nome dos artistas distintos existentes

df_artistas = pd.DataFrame(mdf['nome_artista'].unique(), columns = ['nome_artista'])
df_artistas['nome_artista'] = df_artistas['nome_artista'].str.split('&')
df_artistas = df_artistas.explode('nome_artista')
df_artistas['nome_artista'] = df_artistas['nome_artista'].str.strip().replace(u'\xa0', ' ')
df_artistas.reset_index(drop = True, inplace = True)

# Adicionando ao DF o link do perfil de cada artista no Genius 

df_artistas['link_musicas_artista'] = df_artistas['nome_artista'].apply(get_link_musicas_artista)

# Exibe novo DataFrame dos artistas

df_artistas

,nome_artista,link_musicas_artista
0,ANAVITÓRIA,https://genius.com/artists/Anavitoria/songs
1,Milton Nascimento,https://genius.com/artists/Milton-nascimento/s...
2,Chico Buarque,https://genius.com/artists/Chico-buarque/songs
3,Silva,https://genius.com/artists/Silva/songs
4,Anitta,https://genius.com/artists/Anitta/songs
...,...,...
270,¡IT’S ALESSANDRO!,https://genius.com/artists/!it's-alessandro!/s...
271,Gonzaguinha,https://genius.com/artists/Gonzaguinha/songs
272,Maria Rita,https://genius.com/artists/Maria-rita/songs
273,Marcelo Jeneci,https://genius.com/artists/Marcelo-jeneci/songs


In [6]:
# Para persistir os novos dados e índices de parada

dicionario_musicas = {'indice_artista': [],
                     'indice_musica': [],
                     'titulo_musica': [],
                     'nome_artista': [],
                     'letra': [],
                     'album': [],
                     'data_lancamento': [],
                     'tags': [],
                     'identificador' : []}

musicas_df = pd.DataFrame.from_dict(dicionario_musicas)
nome_arquivo_musicas = 'musicas_artistas.csv'
indice_artista = 0
indice_musica = 0
if os.path.isfile(nome_arquivo_musicas):
    musicas_df = pd.read_csv(nome_arquivo_musicas)
    if len(musicas_df) > 0:
        indice_artista = list(musicas_df['indice_artista'])[-1]
        indice_musica = list(musicas_df['indice_musica'])[-1]
else:
    musicas_df.to_csv(nome_arquivo_musicas, index=False)

print(indice_artista, indice_musica)

2 0


In [ ]:
# Identificando qual o sistema operacional

sistema_operacional = platform.system()
if sistema_operacional == 'Linux':
    caminho_webdriver = os.path.join('chromedriver_linux64', 'chromedriver')
elif sistema_operacional == 'Windows':
    caminho_webdriver = os.path.join('chromedriver_win64', 'chromedriver.exe')

# Criando o WebDriver
    
servico = Service(executable_path=caminho_webdriver)
opcoes = webdriver.ChromeOptions()
navegador = webdriver.Chrome(service = servico)
espera_navegador = WebDriverWait(navegador, 10)
identificadores = list(musicas_df['identificador'])

for i in range(indice_artista, len(df_artistas)):
    link_musicas_artista = df_artistas['link_musicas_artista'].iloc[i]
    navegador.get(link_musicas_artista)
    
    while True:
        navegador.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        if not (len(navegador.find_elements(By.XPATH, "//div[contains(@class, 'PlaceholderSpinner')]"))) > 0:
            break
    
    if len(navegador.find_elements(By.CLASS_NAME, 'render_404-headline')) > 0:
        continue
    
    musicas_elementos = navegador.find_elements(By.XPATH, '//a[contains(@class, "ListItem__Link")]')
        
    for j in range(indice_musica, len(musicas_elementos)):

        musica = musicas_elementos[j]
        link_letra = musica.get_attribute("href")
        navegador.execute_script(f'window.open("{link_letra}","_blank");')
        navegador.switch_to.window(navegador.window_handles[1])

        # Captura informações importantes da música
        
        titulo_musica_elemento = navegador.find_element(By.XPATH, '//span[contains(@class, "SongHeaderdesktop__HiddenMask")]')
        titulo_musica = titulo_musica_elemento.get_attribute("innerText")
        
        nome_artista_elemento = navegador.find_element(By.XPATH, '//a[contains(@class, "HeaderArtistAndTracklistdesktop__Artist")]')
        nome_artista = nome_artista_elemento.get_attribute("innerText")
        
        identificador = f'{titulo_musica} {nome_artista}'
        
        if identificador not in identificadores: 
            
            letra = ''
            try:
                identificador_containers_letra_id = '//div[@data-lyrics-container="true"]'
                containers_letra = espera_navegador.until(EC.presence_of_all_elements_located((By.XPATH, identificador_containers_letra_id)))
                for container in containers_letra:
                    letra_atual = container.get_attribute("innerText")
                    letra += letra_atual
            except Exception as e:
                print(f'Erro na letra: {str(e)}')

            album = ''
            try:
                album = navegador.find_element(By.XPATH, "//a[contains(@class, 'PrimaryAlbum')]").get_attribute("innerText")
            except Exception as e:
                print(f'Erro no nome do álbum: {str(e)}')

            data_lancamento = ''
            try:
                identificador_data_lancamento = "//span[contains(@class, 'MetadataStats')]"
                elementos_data_lancamento = navegador.find_elements(By.XPATH, identificador_data_lancamento)
                data_lancamento = elementos_data_lancamento[0].get_attribute("innerText")
            except Exception as e:
                print(f'Erro na data de lançamento: {str(e)}')

            tags = ''
            try:
                identificador_tags = "//a[contains(@class, 'SongTags__Tag')]"
                elementos_tags = navegador.find_elements(By.XPATH, identificador_tags)
                tags = ','.join([tag.get_attribute("innerText") for tag in elementos_tags])
            except Exception as e:
                print(f'Erro nas tags: {str(e)}')
            
            i_copia = i
            j_copia = j + 1
            if j ==  len(musicas_elementos) - 1:
                i_copia += 1
                j_copia = 0
                
            nova_linha = [i_copia, j_copia, titulo_musica, nome_artista, letra, album, data_lancamento, tags, identificador]
            df_nova_linha = pd.DataFrame([nova_linha])
            df_nova_linha.to_csv(nome_arquivo_musicas, mode='a', header=False, index=False)
            identificadores.append(identificador)

        navegador.close()
        navegador.switch_to.window(navegador.window_handles[0])